In [311]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np
import random

# data 읽기
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'data/u.user', sep='|', names=users_cols, encoding='latin-1')

watch_cols = ['user_id', 'movie_id', 'watch_hist_time']
watches = pd.read_csv(
    'data/u.watch', sep='\t', names=watch_cols, encoding='latin-1')

search_cols = ['user', 'search_hist']
searches = pd.read_csv(
    'data/u.search', sep='\t', names=search_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'data/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
watches["movie_id"] = watches["movie_id"].apply(lambda x: str(x-1))
watches["user_id"] = watches["user_id"].apply(lambda x: str(x-1))
searches["user_id"] = searches["user"].apply(lambda x: str(x-1))
# example_age 추가
movies['example_age'] = (pd.to_datetime("now") - pd.to_datetime(movies['release_date']))\
            /np.timedelta64(1,'D') 
movies = normalize_col(movies,'example_age')

watches = normalize_col(watches,'watch_hist_time')


# data 합치기
data = watches.merge(movies, on='movie_id').merge(users, on='user_id')
data['user_id']=data['user_id'].astype(int)
data['movie_id']=data['movie_id'].astype(int)
data = data.set_index(['user_id']).sort_index()
data = data.reset_index()
data['movie_name']=data['title'].str[:-6] # 년도 부분 자르기


def normalize_col(df,col_name):
    df[col_name] = (df[col_name] - df[col_name].min()) / (df[col_name].max() - df[col_name].min())
    return df



# 유저 인덱스 인코딩
user_ids = data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

# 영화 인덱스 인코딩
movie_ids = data["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

# 영화 제목 인코딩
title_ids = data["title"].unique().tolist()
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

# 인덱스 번호로 바꾸기
data["user"] = data["user_id"].map(user2user_encoded)
data["movie"] = data["movie_id"].map(movie2movie_encoded)
data["title_d"] = data["title"].map(title2title_encoded)


watch_hist = data.groupby(['user'])['movie_id'].apply(list).reset_index()
search_hist = searches.groupby(['user'])['search_hist'].apply(list).reset_index()
watch_hist_time = data.groupby(['user'])['watch_hist_time'].apply(list).reset_index()
example_age = data.groupby(['user'])['example_age'].apply(list).reset_index()

user_video_list = data.pivot(index='user_id', columns='movie_id', values='movie').reset_index()
user_video_list.fillna(data["movie_id"].max()+1, inplace=True)

sample_data=data[['user','occupation','sex']]
sample_data=sample_data.reset_index()
sample_data = sample_data.drop('index',axis=1)
sample_data = sample_data.drop_duplicates()

user_movie_list = pd.merge(sample_data,watch_hist, how= 'left')
user_movie_list = pd.merge(user_movie_list,watch_hist_time, how='left')
user_movie_list = pd.merge(user_movie_list,search_hist, how='left')
user_movie_list = pd.merge(user_movie_list,example_age, how='left')
user_movie_list['search_hist'] = user_movie_list['search_hist'].apply(lambda x: x if type(x) is list else []) # NaN 처리
user_movie_list['predict_labels'] = user_movie_list['movie_id'].apply(lambda x: int(random.uniform(0,data["movie"].max()))) #label을 마지막 값으로..



train_data = user_movie_list[(user_movie_list.user >= 1)&
                                  (user_movie_list.user <= 5)]
test_data = user_movie_list[(user_movie_list.user >= 6)&
                                  (user_title_list.user <= 10)]





In [312]:
movies # 영화 정보 데이터

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,example_age
0,0,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1995,1.000000
1,1,GoldenEye (2011),01-Jan-2011,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,1,0,0,2011,0.359982
2,2,Four Rooms (2020),01-Jan-2020,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2020,0.000000
3,3,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1995,1.000000
4,4,Copycat (1998),01-Jan-1998,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,1998,0.879969
5,5,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1995,1.000000
6,6,Twelve Monkeys (2018),01-Jan-2018,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,2018,0.079947
7,7,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,0,0,0,0,0,1995,1.000000
8,8,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1995,1.000000
9,9,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1996,0.957726


In [313]:
train_data # train data

,user,occupation,sex,movie_id,watch_hist_time,search_hist,example_age,predict_labels
1,1,other,F,"[1, 6]","[0.018896811163116225, 0.174139363982328]","['Shorty', 'Twelve Monkeys']","[0.3599824772752163, 0.07994743182564888]",0
2,2,writer,M,"[2, 7]","[0.1887056559205634, 0.2183194085997988]",['Copycat'],"[0.0, 1.0]",2
3,3,technician,M,[7],[1.0],['Twelve Monkeys'],[1.0],3
4,4,other,F,[3],[0.029307554350203404],[],[1.0],8
5,5,executive,M,[5],[0.17120860854730763],[],[1.0],8


In [314]:
test_data # test data

,user,occupation,sex,movie_id,watch_hist_time,search_hist,example_age,predict_labels
6,6,administrator,M,"[1, 7]","[0.4177420060364813, 0.1887056559205634]",[],"[0.3599824772752163, 1.0]",3
7,7,administrator,M,"[6, 9, 0]","[0.21398888937491797, 0.174139363982328, 0.041...",['Shorty'],"[0.07994743182564888, 0.9577264264593144, 1.0]",7
8,8,student,M,[0],[0.22129390665325227],[],[1.0],5
9,9,lawyer,M,[0],[0.0],[],[1.0],8


In [326]:
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=data["movie"].max() + 3
LEARNING_RATE = 0.003

In [327]:
import tensorflow as tf
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}
    
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask

In [328]:
#---inputs
import tensorflow as tf
import datetime
import os
input_watch_hist = tf.keras.Input(shape=(None, ), name='watch_hist')
input_watch_hist_time = tf.keras.layers.Input(shape=(None,), name='watch_hist_time')
# input_search_hist = tf.keras.layers.Input(shape=(None,), name='search_hist')
input_example_age = tf.keras.Input(shape=(None, ), name='example_age')
# input_occupation = tf.keras.Input(shape=(None, ), name='occupation')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_watch_hist)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_watch_embeddings = labels_embedding_layer(input_watch_hist_time)
l2_norm_watched = l2_norm_1(labels_watch_embeddings)
avg_watched = avg_embeddings(l2_norm_watched)

# labels_search_embeddings = labels_embedding_layer(input_search_hist)
# l2_norm_searched = l2_norm_1(labels_search_embeddings)
# avg_searched = avg_embeddings(l2_norm_searched)

labels_example_age_embeddings = labels_embedding_layer(input_example_age)
l2_norm_example_age = l2_norm_1(labels_example_age_embeddings)
avg_example_age = avg_embeddings(l2_norm_example_age)

# labels_occupation_embeddings = labels_embedding_layer(input_occupation)
# l2_norm_occupation = l2_norm_1(labels_occupation_embeddings)
# avg__occupation = avg_embeddings(l2_norm_occupation)


print(avg_features)
print(avg_watched)
# print(avg_searched)
print(avg_example_age)
# print(input_occupation)

# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_watched,
#                                                      avg_searched,
                                                     avg_example_age
#                                                      avg__occupation
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_watch_hist, 
            input_watch_hist_time, 
#             input_search_hist,
            input_example_age,
#             input_occupation,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

model.summary()

Tensor("aggregate_embeddings/PartitionedCall_99:0", shape=(None, 16), dtype=float32)
Tensor("aggregate_embeddings/PartitionedCall_100:0", shape=(None, 16), dtype=float32)
Tensor("aggregate_embeddings/PartitionedCall_101:0", shape=(None, 16), dtype=float32)
Model: "functional_39"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
watch_hist (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
watch_hist_time (InputLayer)    [(None, None)]       0                                            
__________________________________________________________________________________________________
example_age (InputLayer)        [(None, None)]       0                                            
___________________________

In [329]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['movie_id']),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['watch_hist_time']),
#            tf.keras.preprocessing.sequence.pad_sequences(train_data['search_hist']),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['example_age'])
#            tf.keras.preprocessing.sequence.pad_sequences(train_data['occupation']),
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=100)




Epoch 1/100


InvalidArgumentError:  indices[0] = 9 is not in [0, 5)
	 [[{{node functional_39/aggregate_embeddings/PartitionedCall_1/RaggedMask/boolean_mask/GatherV2}}]] [Op:__inference_train_function_41421]

Function call stack:
train_function


In [ ]:
train_data


In [ ]:
data

In [330]:
print(tf.keras.preprocessing.sequence.pad_sequences(train_data['watch_hist_time']))

[[0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]]
